<!-- Auto table of contents -->
<h1 class='tocIgnore'>Générateur semi-automatique de page HTML "quizz oiseaux"</h1>
<p>(application des "Oiseaux à l'Oreille" : les quizz <b>2020</b>)</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table des matières</h2>
  <div id="toc"></div>
</div>

In [1]:
%%javascript
$.getScript('../../ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Imports

In [2]:
import os
import datetime as dt

from collections import OrderedDict as odict

import requests

from IPython.display import HTML

In [3]:
import sys
sys.path.append('..')

import quizz

import importlib as implib

In [4]:
# Les étapes : les mêmes pour tous les quizz et tous les exercices.
KEtapes = \
[quizz.DescripteurEtape(index=1, id='lancement', titre='Les enregistrements à analyser et les 1er indices'),
 quizz.DescripteurEtape(index=2, id='indices2', titre='Quelques indices à mi-parcours'),
 quizz.DescripteurEtape(index=3, id='reponse', titre='Les réponses : analyse détaillée des enregistrements')]

# Commutateur de publication locale ou internet

In [5]:
KDossierQuizz = '.'

pubLocale = True

dossierSons = os.path.join(KDossierQuizz, 'enregistrements')

if pubLocale:
    
    # 1) Locale
    urlDossierSons = dossierSons.replace('\\', '/') # Jouer les sons en lisant les fichiers locaux.
    urlDossierAnnexes = os.path.join(KDossierQuizz, 'fichiers').replace('\\', '/') # idem, ailleurs

else:
    
    # 2) Internet
    urlDossierSons = '.' # Jouer depuis XenoCanto si enregistrement en vient, et n'a pas eté modifié ('-extrait'),
                         # sinon depuis le dossier de la page HTML
    urlDossierAnnexes = '.' # Fichiers annexes sur mon site, dans le dossier de la page HTML

# Préambule et remerciements

In [6]:
# texte syntaxe markdown
KPreambule = \
"""
L'été avance, les migrateurs commencent déjà à passer par chez nous ...
il est temps de reprendre les quizz d'ornithoreille !

Exactement sur le même modèle que les années précédentes, alors si vous connaissez déjà,
vous pouvez vous épargner le blabla qui suit, et passer directement au <a href="#Aout">quiz d'août</a> :-)

Chacun de ces exercices, nommé comme le mois de 2020 durant lequel il a est lancé, se déroule en *3 étapes*,
publiée chacune 3 semaines après la précédente :
1. la première, le **lancement**, vous donne accès aux enregistrements à analyser
   et à quelques premiers indices pour démarrer,
2. la deuxième, à **mi-parcours**, vous donne des indices plus conséquents, pour confirmer ou pas vos premières idées
   et / ou pour aller plus loin en ciblant quelques séquences plus difficiles,
3. la troisième et dernière, pour terminer, vous donne la "**solution**", en détaillant pour chaque enregistrement
   et chaque espèce, les différents types de chants et cris entendus, et les instants précis concernés.

Pour accéder à l'étape suivante, lorsqu'elle est publiée, il vous suffit de cliquer sur le **petit oeil vert**
<img height="16" src="{{dossierAttache}}/fa-eye-regular.svg" alt="Montrer"/>
situé sous le titre de cette dernière ; bien sûr, tant qu'elle n'est pas publiée, vous ne verrez rien de plus ;-)

Chaque quiz porte sur **plusieurs enregistrements** (en général 3) indépendants, pour varier les milieux et les espèces.

Pour chaque enregistrement (répété à chaque étape), vous trouverez :
* un **lecteur audio intégré**, pour l'écouter, le ré-écouter ... etc, autant de fois que nécessaire,
* un **lien de téléchargement**, pour pouvoir l'analyser tranquillement et précisément
sur votre tablette ou ordinateur, avec un lecteur plus pratique notamment en ce qui concerne
le positionnement temporel ; pour télécharger l'enregistrement, il vous suffit de cliquer sur le bouton
<img height="16" src="{{dossierAttache}}/fa-download-solid.svg" alt="Télécharger"/>
(à moins qu'il ne faille un clic droit, suivi de "enregistrer la cible du / le lien sous ..."),
* le nom de l'**auteur** de l'enregistrement et les détails de sa **licence** de publication,
* à la dernière étape, si disponible, le lien vers la **page du site source** de l'enregistrement
  (xeno-canto.org en général).

Pub. <a href="http://audacity.fr/" target="_blank">Audacity</a> est - entre autres - un lecteur audio très pratique
et précis pour la gestion du temps : essayez-le, je vous le conseille :-) ; il est disponible gratuitement
et en toute sécurité (logiciel Open Source) sous Windows, Mac OS ou Linux (mais pas Android ou i-OS :-(.

Chaque quiz permet à chacun de jouer **quel que soit son niveau**, du débutant au plus expérimenté :
si vous ne trouvez pas tout de A à Z, c'est **normal**, certains enregistrements sont plus difficiles
que d'autres, et à dessein : à vous, pour chaque concert, de trouver **votre défi personnel** ;
par exemple, au delà des espèces que vous avez reconnues rapidement, en choississant 1 chant (ou un cri), ou 2,
que vous ne reconnaissez pas, mais pour lequel les indices à mi-parcours vous orientent suffisamment,
et en cherchant vous-même la solution ailleurs (les <a href="http://jpmeuret.free.fr/nature.html" target="_blank">
supports de formation</a>, bien sûr, mais aussi des CD, des applis dédiées, des livres, des sites internet
comme <a href="http://www.oiseaux.net/oiseaux/france.html" target="_blank">oiseaux.net</a>,
<a href="https://www.xeno-canto.org/" target="_blank">xeno-canto.org</a>, des amis, et surtout les oiseaux réels
lors de vos ballades, dans votre jardin ...).

Ce défi personnel, que vous seul(e) pouvez ajuster à vos connaissances actuelles, vous permettra
de progresser, en vous poussant **au delà de ce que vous savez** déjà ; mais il ne devra pas être **trop ambitieux**
d'un coup, car l'important, c'est d'éviter de dépasser le niveau de frustration qui fait renoncer,
et de rester dans celui qui pousse à avancer :-)

Et puis gardez confiance : l'exercice "en intérieur" sur ordinateur n'est pas simple, tout le monde le dit :
* on n'est pas immergé dans le milieu naturel,
* la qualité du son n'est pas toujours très bonne (enregistrement, casque, haut-parleurs, ...),
* on ne peut pas tourner la tête pour mieux entendre ... même en stéréo, on manque de repères dans l'espace,
* ...

Vous constaterez vite, lors de vos ballades "dans le monde réel", avec les espèces que vous connaissez,
que vous êtes plus à l'aise à l'extérieur : meilleur son, moins de bruit de fond, plus de clarté,
plus de profondeur, ... c'est normal, aucun intermédiaire entre les oiseaux et vos oreilles !

... Mais il y a un hic : en nature, il n'y a pas de bouton "Rejouer" ;-)

Dernier détail, mais d'importance, je compte sur vous pour me les signaler les **coquilles** et **erreurs**
qui se sont inévitablement glissées dans ce qui suit, à me faire part de votre **désaccord** éventuel
sur telle ou telle affirmation ou identification, ou encore à me signaler toute chose que j'aurais "**zappée**"
dans les enregistrements :-)

Amusez-vous bien !
"""

In [7]:
KAttribsEtMercis = \
"""
Merci aussi à <a href="https://www.faune-auvergne.org/index.php?m_id=53&id=705" target="_blank">
Monique et Jacques Lombardy</a> et Yvon Le Corre (pas le peintre, l'ornithologue, breton aussi :-)
pour leurs splendides photos de la grive musicienne et du bruant ortolan en haut de page (tous droits réservés).
"""

# Merci aussi à Annie, Camille, Dominique, Florence, Monique et Sylvain pour leurs questions et remarques
# bien utiles pour affiner les indices de mi-parcours et les listes de confusions possibles, et pour leurs trouvailles
# qui m'ont permis de creuser d'avantage les analyses et de compléter ainsi les listes d'espèces avec celles
# que j'avais zappées :-)

# Quizz d'août

In [8]:
KAoutIntro = """
  Pour commencer, quatre concerts plutôt calmes (sauf le dernier), tous enregistrés en Auvergne, voire dans le Puy-de-Dôme ...
  histoire de nous rappeler - si besoin - ces heures de confinement du printemps 2020 ;-)
"""

In [9]:
KAoutBocagePrairiesMoyenneAltitude = quizz.DescripteurExercice(

  index='a',
  id='BocagePrairiesMoyenneAltitude',
  titre="Bocage et prairies à moyenne altitude",
  lieu="La Batrande (est), Aydat, Puy-de-Dôme", altitude=930,
  date="2 avril 2020", heure="le matin", duree="58 s",
  auteur="Anne Citron",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Bocage en déprise et prairies sur plateau de moyenne altitude""",
  fichierSon="200402-hhmm-AydatBatrandeE-AC.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Un premier concert campagnard, clair et ensoleillé, peu chargé, sans doute assez tôt le matin
      (certains oiseaux ont encore la voix enrouée ;-).
      
      5 espèces a priori, communes à très communes, dont 3 chantent véritablement.
      """,
          
    indices2="""
      """,
          
    reponse="""
      """))

HTML(KAoutBocagePrairiesMoyenneAltitude.lecteurHtml(urlDossierSons))

In [10]:
KAoutVergerVillageChateau = quizz.DescripteurExercice(

  index='b',
  id='StAgoulinGirauds',
  titre="Un verger près du parc du château",
  lieu="Les Girauds, Saint-Agoulin, Puy-de-Dôme", altitude=500,
  date="19 avril 2020", heure="fin d'après-midi", duree="40 s",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Verger, jardins dans village à proximité d'un parc boisé.""",
  fichierSon="200419-1757-306-StAgoulinGirauds-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Il est juste 6h du soir à l'horloge de l'église, mais les cloches qui les sonnent ne semblent pas
      perturber le moins du monde les solistes de ce concert plutôt calme et aéré ... pas plus que la petite pluie
      qui se renforce un peu à la fin.
      
      6 espèces très communes chantent, et une crie.
      
      Si vous vous ennuyez, essayez donc de compter les chanteurs de chaque espèce :-)
      """,
          
    indices2="""
    """,
          
    reponse="""
      """))

HTML(KAoutVergerVillageChateau.lecteurHtml(urlDossierSons))

In [11]:
KAoutGrandesCulturesLimagnePerchee = quizz.DescripteurExercice(

  index='c',
  id='GrandesCulturesLimagnePerchee',
  titre="Quelques ilôts dans la mer des grandes cultures de la limagne 'perchée'",
  lieu="Nord-ouest de Bens, Artonne, Puy-de-Dôme", altitude=480,
  date="7 juin 2020", heure="10h45", duree="1 mn",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Grandes cultures, haies et broussailles en bordure de plateau calcaire.""",
  fichierSon="200607-1045-360-ArtonneBens-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      La matinée est bien avancée déjà en cette fin de printemps , mais quelques acharnés s'acharnent encore,
      toujours un peu les mêmes dans ces grandes cultures vallonnées parfois interrompues par un chemin, un noyer,
      une haie, ou des broussailles, à la faveur d'un rideau agraire ou d'une crête incultivable.
      
      5 espèces chantent, dont 1 plus du tout courante en Auvergne, snif, et une autre abondante, toujours en forme,
      rarement discrète et particulièrement en verve ici ; sans doute une 6ème très loin également.
      
      Et un criquet pas encore très chaud apparemment.""",
          
    indices2="""
      """,
          
    reponse="""
        """))

HTML(KAoutGrandesCulturesLimagnePerchee.lecteurHtml(urlDossierSons))

In [12]:
KAoutBocageForetPreBassesCombrailles = quizz.DescripteurExercice(

  index='d',
  id='BocageBoisPreBassesCombrailles',
  titre="Bocage et bois dans les pré-Basses-Combrailles",
  lieu="Sous les Bardons, Artonne, Puy-de-Dôme", altitude=450,
  date="24 mars 2019", heure="8 heures du matin", duree="52 s",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Bocage et bois sur les pentes d'un vallon frais, dans le nord du Puy-de-Dôme.""",
  fichierSon="190324-0814-116-ArtonneSousLesBardons-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Et pour ce dernier concert, plus chargé, retour en tout début de printemps : 1 seul migrateur est arrivé ici.
      
      Un grand bois de vieux chênes couvre la pente de l'autre côté du vallon ; un ruisseau coule en bas.
      
      11 espèces très communes.  
      """,
          
    indices2="""
      """,
          
    reponse="""
        """))

HTML(KAoutBocageForetPreBassesCombrailles.lecteurHtml(urlDossierSons))

# Génération de la page

In [13]:
KQuizz = \
[quizz.DescripteurQuizz(id='Aout', titre='Quizz d\'août',
                        intro=KAoutIntro,
                        publier=dict(lancement=True, indices2=False, reponse=False),
                        exercices=[KAoutBocagePrairiesMoyenneAltitude, KAoutVergerVillageChateau,
                                   KAoutGrandesCulturesLimagnePerchee, KAoutBocageForetPreBassesCombrailles],
                        anecdotes=[],
                        effort=dict(infrastructure=0, lancement=8, indices2=0, reponse=0, anecdotes=0, courrier=0)),
]

nomFicCible = \
    quizz.buildHtmlPage(titre="Les Oiseaux à l'Oreille : Quizzzzz 2020",
                        sousTitre="C'est la reprise, fini les vacances : à vos oreilles, prêt(e)s, écoutez !",
                        description="Quizz 2020 d'application de la formation \"Les Oiseaux à l'Oreille\"",
                        motsClef='auvergne, forêt, bocage, montagne, plaine, '
                                 'bruant, fauvette, alouette, pouillot, pipit',
                        preambule=KPreambule, quizz=KQuizz, etapes=KEtapes, attribsEtMercis=KAttribsEtMercis,
                        dossierAttache=urlDossierAnnexes, dossierSons=urlDossierSons,
                        images=dict(imgTMat=[dict(img='GriMus-MJLombardy-20130304a.jpg',
                                                  legend='Grive musicienne'),
                                             dict(img='BruOrt2-YLeCorre-201804.jpg',
                                                  legend='Bruant ortolan')]),
                        notebook='Quizz2020.ipynb', prefixeFicCible='quizz-2020',
                        effort=sum(sum(q.effort.values()) for q in KQuizz))

HTML("""<p>Quizz 2020 <a href='{nomFic}' target="_blank">{nomFic}</a> (nouvel onglet)</p>""".format(nomFic=nomFicCible))

In [14]:
raise Exception("On s'arrête ici !")

Exception: On s'arrête ici !

In [ ]:
implib.reload(module=quizz)

In [ ]:
HTML(quizz.infosEnregXC(nr=471826))